In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Neural network for a single observation
x = np.array(df.iloc[0, 1:])
y = df.iloc[0, 0]

In [28]:
# Generate initial weights and biases: Choose all to be 1
hl_1_size = 3
W = np.array([np.ones(len(x)) for _ in range(hl_1_size)])
b = np.ones(hl_1_size)

In [33]:
# ReLU activation function
def relu(v):
    return np.array([max(0, i) for i in v])

In [36]:
z = (W @ x) + b
a = relu(z)